#### 配置和连接Ganache

In [1]:
from web3 import Web3
from solcx import install_solc, set_solc_version, compile_files
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import random

# 连接 Ganache
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
assert w3.is_connected(), "请确保 Ganache 已启动"

# 安装 Solidity 编译器
install_solc('0.8.17')  # 安装与合约代码匹配的版本
set_solc_version('0.8.17')

w3.eth.default_account = w3.eth.accounts[0]

print(f"连接状态: {w3.is_connected()}")
print(f"默认账户: {w3.eth.default_account}")
print(f"初始余额: {Web3.from_wei(w3.eth.get_balance(w3.eth.accounts[0]), 'ether')} ETH")

连接状态: True
默认账户: 0x9be4889f44c72415c43a9c18e71C87448971Bc53
初始余额: 1000 ETH


### 读取solidity file，编译合约的函数

In [2]:
from solcx import compile_files
from IPython.display import display, Markdown

def read_compile_contract(solidity_file, contract_name=None):
    """
    增强版合约编译函数
    :param solidity_file: Solidity文件路径
    :param contract_name: 明确指定要编译的合约名称（可选）
    :return: (abi, bytecode_bytes)
    """
    # 读取合约源码
    with open(solidity_file, 'r', encoding='utf-8') as f:
        contract_source = f.read()
    
    # 显示源码（可选）
    display(Markdown(f"### 正在编译 {solidity_file}"))
    # print(contract_source)  # 调试时可取消注释
    
    # 执行编译（明确指定编译器版本）
    compiled = compile_files(
        [solidity_file],
        output_values=['abi', 'bin'],
        solc_version='0.8.17'  # 强制指定编译器版本
    )
    
    # 定位目标合约（修正关键问题）
    target_contract_key = None
    for contract_key in compiled.keys():
        # 格式为 "<stdin>:ContractName" 或 "filename.sol:ContractName"
        if contract_name:
            if contract_key.endswith(f":{contract_name}"):
                target_contract_key = contract_key
                break
        else:
            # 默认选择第一个合约
            target_contract_key = next(iter(compiled.keys()))
            break
            
    if not target_contract_key:
        available_contracts = list(compiled.keys())
        raise ValueError(f"未找到合约 '{contract_name}'，可用合约: {available_contracts}")
    
    # 提取接口
    contract_interface = compiled[target_contract_key]
    
    # 处理字节码转换（修正关键问题）
    bytecode_hex = contract_interface['bin']
    if bytecode_hex.startswith('0x'):
        bytecode_hex = bytecode_hex[2:]  # 去除0x前缀
    try:
        bytecode_bytes = bytes.fromhex(bytecode_hex)
    except ValueError as e:
        raise ValueError(f"无效的字节码格式: {str(e)}") from e
    
    # 显示编译摘要
    display(Markdown("#### 编译结果"))
    print(f"合约名称: {target_contract_key.split(':')[-1]}")
    print(f"ABI长度: {len(contract_interface['abi'])} 项")
    print(f"字节码长度: {len(bytecode_bytes)} 字节")
    
    return contract_interface['abi'], bytecode_bytes

#### 读取和编译incentive和consensus合约

In [4]:
incentive_solidity_file = '../contracts/FLIncentiveOptimized.sol'
consensus_solidity_file = '../contracts/FLConsensus.sol'
# 编译合约
incentive_abi, incentive_bytecode = read_compile_contract(incentive_solidity_file)
consensus_abi, consensus_bytecode = read_compile_contract(consensus_solidity_file)

# 示例输出：
# [验证] 字节码类型: <class 'bytes'>
# [验证] 字节码长度: 1234

### 正在编译 ../contracts/FLIncentiveOptimized.sol

#### 编译结果

合约名称: FLIncentiveOptimized
ABI长度: 33 项
字节码长度: 15447 字节


### 正在编译 ../contracts/FLConsensus.sol

#### 编译结果

合约名称: FLConsensus
ABI长度: 23 项
字节码长度: 9365 字节


### 部署+验证智能合约



#### incentive合约

In [5]:
# 获取区块链当前时间
current_block = w3.eth.get_block('latest')
blockchain_time = current_block.timestamp
# 部署参数配置
constructor_args = {
    "_deadline": int(time.time()) + 86400,  # 24小时后截止
    "_gracePeriod": 7200,     # 2小时宽限期
    "_claimWindow": 604800,   # 7天领取期
    "_stakeMatrix": [
        [Web3.to_wei(0.1, 'ether'), Web3.to_wei(0.2, 'ether'), Web3.to_wei(0.3, 'ether')],
        [Web3.to_wei(0.4, 'ether'), Web3.to_wei(0.5, 'ether'), Web3.to_wei(0.6, 'ether')],
        [Web3.to_wei(0.7, 'ether'), Web3.to_wei(0.8, 'ether'), Web3.to_wei(0.9, 'ether')]
    ],
    "_multiplierMatrix": [
        [100, 200, 300],
        [400, 500, 600],
        [700, 800, 900]
    ],
    "_minParticipants": 5  # 新增参数
}

# 部署合约
def deploy_incentive_contract():
    try:
        contract = w3.eth.contract(
            abi=incentive_abi,
            bytecode=incentive_bytecode
        )
        
        # 估算Gas
        estimated_gas = contract.constructor(
            constructor_args["_deadline"],
            constructor_args["_gracePeriod"],
            constructor_args["_claimWindow"],
            constructor_args["_stakeMatrix"],
            constructor_args["_multiplierMatrix"],
            constructor_args["_minParticipants"]  # 新增参数
        ).estimate_gas({
            'from': w3.eth.default_account,
            'value': Web3.to_wei(10, 'ether')
        })
        
        # 发送交易
        tx_hash = contract.constructor(
            constructor_args["_deadline"],
            constructor_args["_gracePeriod"],
            constructor_args["_claimWindow"],
            constructor_args["_stakeMatrix"],
            constructor_args["_multiplierMatrix"],
            constructor_args["_minParticipants"]  # 新增参数
        ).transact({
            'from': w3.eth.default_account,
            'value': Web3.to_wei(10, 'ether'),
            'gas': int(estimated_gas * 1.2)
        })
        
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        incentive = w3.eth.contract(address=receipt.contractAddress, abi=incentive_abi)
        # return receipt.contractAddress
        return incentive
    
    except Exception as e:
        print(f"部署失败: {str(e)}")
        return None

# 执行部署
incentive = deploy_incentive_contract()


# 验证部署
print(f"\n🔍 激励合约验证 ({incentive_solidity_file})")
print(f"地址: {incentive.address}")
print(f"字节码长度: {len(w3.eth.get_code(incentive.address))} bytes (应 >2000)")
print(f"初始资金: {Web3.from_wei(incentive.functions.currentPoolBalance().call(), 'ether')} ETH")
print(f"当前阶段: {['ACTIVE', 'GRACE', 'CLOSED'][incentive.functions.currentPhase().call()]}")




🔍 激励合约验证 (../contracts/FLIncentiveOptimized.sol)
地址: 0x62E07208aCA6CE4C1Cf20cc4c96a6a2F5993E3d4
字节码长度: 11946 bytes (应 >2000)
初始资金: 10 ETH
当前阶段: ACTIVE


##### 通过注册参与者来测试incentive

In [6]:
import random
import pandas as pd
from web3 import Web3
from IPython.display import display, Markdown

def register_users(n_users, incentive_contract):
    results = []
    valid_accounts = w3.eth.accounts[1:]  # 确保这是正确的账户切片

    for i in range(n_users):
        # 修正点：将r['user']改为r['用户地址']
        available_accounts = [a for a in valid_accounts if a not in [r['用户地址'] for r in results]]
        if not available_accounts:
            raise ValueError("可用账户不足，无法继续注册")
        user = random.choice(available_accounts)
        
        # 生成随机类别（0-2）
        local_data_cat = random.randint(0, 2)
        local_resource_cat = random.randint(0, 2)
        
        # 确保不超过合约允许的最大类别
        data_cat = min(local_data_cat, 2)
        resource_cat = min(local_resource_cat, 2)
        
        # 获取所需质押金额
        stake = incentive_contract.functions.stakeMatrix(data_cat, resource_cat).call()
        
        try:
            # 发送注册交易
            tx_hash = incentive_contract.functions.join(
                data_cat,
                resource_cat
            ).transact({
                'from': user,
                'value': stake
            })
            receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            
            # 验证交易状态
            assert receipt.status == 1, "交易失败但未抛出异常"
            
            # 获取链上注册信息
            (stored_dcat, stored_rcat, actual_stake, 
             join_time, is_rewarded, joined) = incentive_contract.functions.participants(user).call()
            
            # 记录结果
            results.append({
                '用户地址': user,
                '输入数据类别': f"{local_data_cat}→{data_cat}",
                '输入资源类别': f"{local_resource_cat}→{resource_cat}",
                '质押金额(ETH)': Web3.from_wei(stake, 'ether'),
                '注册状态': '成功',
                '区块高度': receipt.blockNumber,
                '链上数据类别': stored_dcat,
                '链上资源类别': stored_rcat,
                '实际质押(ETH)': Web3.from_wei(actual_stake, 'ether'),
                '是否已奖励': is_rewarded
            })
            
        except Exception as e:
            results.append({
                '用户地址': user,
                '输入数据类别': f"{local_data_cat}→{data_cat}",
                '输入资源类别': f"{local_resource_cat}→{resource_cat}",
                '质押金额(ETH)': Web3.from_wei(stake, 'ether'),
                '注册状态': '失败',
                '错误信息': str(e).split("execution reverted: ")[-1][:50]  # 截取关键错误信息
            })
    
    # 显示结果
    display(Markdown("### 注册结果汇总"))
    df = pd.DataFrame(results)
    display(df)
    
    # 验证链上状态
    display(Markdown("### 链上状态验证"))
    actual_count = incentive_contract.functions.getParticipantCount().call()
    success_count = len(df[df['注册状态'] == '成功'])
    print(f"应注册成功数: {success_count} | 链上记录数: {actual_count}")
    assert actual_count == success_count, "链上参与者数量不匹配"
    
    # 验证随机5个成功注册的详细信息
    sample_users = df[df['注册状态'] == '成功'].sample(min(5, success_count))['用户地址'].tolist()
    for user in sample_users:
        (s_dcat, s_rcat, s_stake, _, _, _) = incentive_contract.functions.participants(user).call()
        record = df[df['用户地址'] == user].iloc[0]
        assert s_dcat == record['链上数据类别'], f"{user}数据类别不匹配"
        assert s_rcat == record['链上资源类别'], f"{user}资源类别不匹配"
        assert s_stake == Web3.to_wei(record['实际质押(ETH)'], 'ether'), f"{user}质押金额不匹配"
    print("✅ 随机抽查验证通过")

# 使用示例（假设已部署incentive合约）
N = 5  # 需要注册的用户数量
print(f"\n=== 开始注册测试，数量：{N} ===")
register_users(N, incentive)

# 显示最终合约状态
display(Markdown("### 最终合约状态"))
print(f"总参与者数: {incentive.functions.getParticipantCount().call()}")
print(f"当前阶段: {['ACTIVE','GRACE','CLOSED'][incentive.functions.currentPhase().call()]}")
print(f"合约余额: {Web3.from_wei(incentive.functions.currentPoolBalance().call(), 'ether')} ETH")


=== 开始注册测试，数量：5 ===


### 注册结果汇总

,用户地址,输入数据类别,输入资源类别,质押金额(ETH),注册状态,区块高度,链上数据类别,链上资源类别,实际质押(ETH),是否已奖励
0,0x639407773853C2d55B285623227Ef5818a4e37b0,2→2,2→2,0.9,成功,2,2,2,0.9,False
1,0xBC504D6f131Ad0a0712Ae0bE1b60564Bb4496B5e,1→1,2→2,0.6,成功,3,1,2,0.6,False
2,0x1486C2decEb64bbC9a82137aF3747241B7fc9C0A,2→2,1→1,0.8,成功,4,2,1,0.8,False
3,0x7A2494055D07f8a70036b02d412A58B7Fe4c0272,1→1,1→1,0.5,成功,5,1,1,0.5,False
4,0x5cCb6Cb14B23E205Ad297E94627D22a7f8f4B134,1→1,2→2,0.6,成功,6,1,2,0.6,False


### 链上状态验证

应注册成功数: 5 | 链上记录数: 5
✅ 随机抽查验证通过


### 最终合约状态

总参与者数: 5
当前阶段: ACTIVE
合约余额: 13.4 ETH


##### 检查合约状态

In [7]:
def check_incentive_state(incentive):
    print("\n=== 激励合约状态检查 ===")
    
    # 基础状态
    phase = incentive.functions.currentPhase().call()
    participants = incentive.functions.getParticipantCount().call()
    min_participants = incentive.functions.minParticipants().call()
    
    print(f"当前阶段: {['ACTIVE','GRACE','CLOSED'][phase]}")
    print(f"注册参与者: {participants}/{min_participants}")
    
    # 关键验证点
    if phase != 2:  # 未处于CLOSED阶段
        print("⚠️ 激励合约未关闭，需推进时间或注册更多参与者")
    elif participants < min_participants:
        print("⚠️ 参与者不足，需重新注册")

check_incentive_state(incentive)


=== 激励合约状态检查 ===
当前阶段: ACTIVE
注册参与者: 5/5
⚠️ 激励合约未关闭，需推进时间或注册更多参与者



=== 尝试强制完成激励合约 ===
强制完成失败: execution reverted: VM Exception while processing transaction: revert Invalid phase


In [9]:
def enhanced_cross_contract_check(consensus, incentive):
    print("\n=== 增强跨合约验证 ===")
    
    try:
        # 1. 直接调用激励合约方法
        phase = incentive.functions.currentPhase().call()
        print(f"直接调用激励合约阶段: {phase} (0=ACTIVE,1=GRACE,2=CLOSED)")
        
        # 2. 通过共识合约间接调用
        consensus_phase = consensus.functions.getIncentivePhase().call()  # 需在合约中添加此视图函数
        print(f"通过共识合约获取阶段: {consensus_phase}")
        
        # 3. 参与者列表验证
        direct_participants = incentive.functions.participantList().call()
        consensus_participants = consensus.functions.getEligibleParticipants().call()
        print(f"直接/间接获取参与者数: {len(direct_participants)} vs {len(consensus_participants)}")
        
    except Exception as e:
        print(f"跨合约验证失败: {str(e)}")
        # 错误细分
        if "Failed to decode output" in str(e):
            print("可能原因: 接口ABI不匹配")
        elif "execution reverted" in str(e):
            print("可能原因: 激励合约状态不满足条件")

# 需在共识合约中添加：
# function getIncentivePhase() external view returns (uint8) {
#     return IFLIncentive(incentiveContract).currentPhase();
# }

#### consensus 合约     

In [8]:
def deploy_and_validate_consensus(incentive_contract, min_participants=5, model_timeout=600):
    """
    部署并验证共识合约
    :param incentive_contract: 已部署的激励合约实例
    :param min_participants: 最小参与者数
    :param model_timeout: 模型超时时间(秒)
    """
    print("\n=== 部署并验证共识合约 ===")
    
    # 前置校验
    assert isinstance(incentive_contract.address, str), "激励合约地址必须为字符串"
    assert w3.is_address(incentive_contract.address), f"无效激励合约地址: {incentive_contract.address}"
    assert w3.eth.get_code(incentive_contract.address) != b'', "激励合约未正确部署"
    
    # 编译共识合约
    # consensus_abi, consensus_bytecode = compile_contract('FLConsensus.sol')
    
    # 部署参数
    constructor_args = {
        "_incentiveContract": incentive_contract.address,
        "_minParticipants": min_participants,
        "_modelTimeout": model_timeout
    }
    
    try:
        # 部署交易
        contract = w3.eth.contract(abi=consensus_abi, bytecode=consensus_bytecode)
        estimated_gas = contract.constructor(
            constructor_args["_incentiveContract"],
            constructor_args["_minParticipants"],
            constructor_args["_modelTimeout"]
        ).estimate_gas({'from': w3.eth.default_account})
        
        tx_hash = contract.constructor(
            constructor_args["_incentiveContract"],
            constructor_args["_minParticipants"],
            constructor_args["_modelTimeout"]
        ).transact({
            'from': w3.eth.default_account,
            'gas': int(estimated_gas * 1.2)
        })
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        consensus = w3.eth.contract(address=receipt.contractAddress, abi=consensus_abi)
        print(f"✅ 共识合约部署成功 地址: {consensus.address}")
        
        # 深度验证
        print("\n=== 深度验证 ===")
        
        # 1. 基础验证
        assert len(w3.eth.get_code(consensus.address)) > 1500, "合约字节码长度异常"
        print(f"字节码验证通过: {len(w3.eth.get_code(consensus.address))} bytes")
        
        # 2. 参数验证
        assert consensus.functions.incentiveContract().call() == incentive_contract.address, "激励合约地址不匹配"
        assert consensus.functions.minParticipants().call() == min_participants, f"最小参与者数应为{min_participants}"
        assert consensus.functions.modelUpdateTimeout().call() == model_timeout, f"超时时间应为{model_timeout}秒"
        print(f"参数验证通过: minParticipants={min_participants}, timeout={model_timeout}s")
        
        # 3. 初始状态验证
        assert consensus.functions.flRound().call() == 0, "初始轮次应为0"
        assert not consensus.functions.isFLActive().call(), "初始应处于非激活状态"
        print("初始状态验证通过: round=0, active=False")
        
        # 4. 跨合约调用验证
        try:
            eligible = consensus.functions.getEligibleParticipants().call()
            print(f"跨合约调用验证: 获取到{len(eligible)}个合格参与者")
        except Exception as e:
            raise AssertionError(f"跨合约调用失败: {str(e)}")
        
        # 5. 时间参数验证
        current_time = w3.eth.get_block('latest').timestamp
        deadline = consensus.functions.currentRoundDeadline().call()
        assert deadline == 0, "未激活时截止时间应为0"
        print("时间参数验证通过: 未激活状态无截止时间")
        
        # 6. 事件日志验证
        event_filter = consensus.events.FLStarted.create_filter(fromBlock=receipt.blockNumber)
        assert len(event_filter.get_all_entries()) == 0, "初始状态不应触发FLStarted事件"
        print("事件日志验证通过: 无异常事件")
        
        return consensus
        
    except Exception as e:
        print(f"❌ 部署验证失败: {str(e)}")
        # 错误诊断
        if "execution reverted" in str(e):
            print("可能原因：构造函数参数校验失败")
        elif "invalid opcode" in str(e):
            print("可能原因：依赖合约状态异常")
        return None

# 使用示例
consensus = deploy_and_validate_consensus(
    incentive_contract=incentive,
    min_participants=5,
    model_timeout=600
)

# 重新部署修正后的共识合约
consensus = deploy_and_validate_consensus(incentive)

# 执行增强验证
enhanced_cross_contract_check(consensus, incentive)

# 输出示例：
# === 增强跨合约验证 ===
# 直接调用激励合约阶段: 2 (CLOSED)
# 通过共识合约获取阶段: 2
# 直接/间接获取参与者数: 5 vs 5



=== 部署并验证共识合约 ===
✅ 共识合约部署成功 地址: 0x17eAC17255f0FBFc6E9239a5f06746A74e000E6E

=== 深度验证 ===
字节码验证通过: 8048 bytes
参数验证通过: minParticipants=5, timeout=600s
初始状态验证通过: round=0, active=False
❌ 部署验证失败: 跨合约调用失败: {'message': 'VM Exception while processing transaction: revert', 'stack': 'CallError: VM Exception while processing transaction: revert\n    at Blockchain.simulateTransaction (/Users/y.wang8uva.nl/.nvm/versions/node/v16.20.2/lib/node_modules/ganache/dist/node/1.js:2:100650)', 'code': -32000, 'name': 'CallError', 'data': '0x'}

=== 部署并验证共识合约 ===
✅ 共识合约部署成功 地址: 0xD56F37882bCEEf1E6B280e7ce6470115B8480436

=== 深度验证 ===
字节码验证通过: 8048 bytes
参数验证通过: minParticipants=5, timeout=600s
初始状态验证通过: round=0, active=False
❌ 部署验证失败: 跨合约调用失败: {'message': 'VM Exception while processing transaction: revert', 'stack': 'CallError: VM Exception while processing transaction: revert\n    at Blockchain.simulateTransaction (/Users/y.wang8uva.nl/.nvm/versions/node/v16.20.2/lib/node_modules/ganache/dist/node/1.js:2:1

In [10]:
def debug_interface_compatibility():
    """接口兼容性深度测试"""
    print("\n=== 接口兼容性测试 ===")
    
    # 获取共识合约中记录的激励地址
    consensus_incentive_addr = consensus.functions.getIncentiveAddress().call()
    print(f"共识合约记录的激励地址: {consensus_incentive_addr}")
    print(f"实际激励合约地址: {incentive.address}")
    assert consensus_incentive_addr == incentive.address
    
    # 方法签名验证
    incentive_funcs = [f for f in incentive.abi if f['type'] == 'function']
    consensus_interface = [f for f in consensus.abi if f['type'] == 'function' and f['name'] in ['currentPhase', 'participantList']]
    
    # 对比关键方法
    mismatch = []
    for func in consensus_interface:
        incentive_match = [f for f in incentive_funcs if f['name'] == func['name']]
        if not incentive_match:
            mismatch.append(f"缺失方法: {func['name']}")
            continue
        if func['outputs'] != incentive_match[0]['outputs']:
            mismatch.append(f"输出不匹配: {func['name']}")
    
    if mismatch:
        print("⚠️ 接口不兼容问题:")
        for issue in mismatch:
            print(f" - {issue}")
    else:
        print("✅ 接口兼容性验证通过")

debug_interface_compatibility()


=== 接口兼容性测试 ===


NameError: name 'consensus' is not defined

In [11]:

# ---------------------------
# 步骤2：部署共识合约
# ---------------------------
# consensus_abi, consensus_bytecode = compile_contract('FLConsensus.sol')
def deploy_consensus_contract(w3, abi, bytecode, incentive.address, min_participants, model_timeout):
    """增强版共识合约部署函数"""
    print("\n=== 部署共识合约 ===")
    
    # 参数验证
    assert w3.is_address(incentive.address), f"无效激励合约地址: {incentive.address}"
    assert min_participants > 0, "最小参与者数必须>0"
    
    try:
        contract = w3.eth.contract(abi=abi, bytecode=bytecode)
        
        # 估算Gas
        estimated_gas = contract.constructor(
            incentive.address,  # 参数顺序必须与合约定义一致
            min_participants,
            model_timeout
        ).estimate_gas({'from': w3.eth.default_account})
        
        # 发送交易
        tx_hash = contract.constructor(
            incentive.address,
            min_participants,
            model_timeout
        ).transact({
            'from': w3.eth.default_account,
            'gas': int(estimated_gas * 1.2)
        })
        
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"✅ 共识合约部署成功 地址: {receipt.contractAddress}")
        return receipt.contractAddress
    
    except Exception as e:
        print(f"❌ 部署失败: {str(e)}")
        # 详细错误分析
        if "invalid address" in str(e).lower():
            print("可能原因：激励合约地址格式错误")
        elif "execution reverted" in str(e).lower():
            print("可能原因：构造函数参数校验失败")
        return None


# 执行部署
consensus = deploy_consensus_contract()   
# consensus = w3.eth.contract(address=consensus_address, abi=consensus_abi)

# 验证部署
print(f"\n🔍 共识合约验证 ({consensus_solidity_file})")
print(f"地址: {consensus.address}")
print(f"字节码长度: {len(w3.eth.get_code(consensus.address))} bytes (应 >2000)")
print(f"初始资金: {Web3.from_wei(consensus.functions.currentPoolBalance().call(), 'ether')} ETH")
# print(f"当前阶段: {['ACTIVE', 'GRACE', 'CLOSED'][incentive.functions.currentPhase().call()]}")



SyntaxError: invalid syntax (303098024.py, line 5)

In [ ]:
def validate_consensus_deployment(consensus_contract, incentive_address):
    """共识合约部署验证"""
    print("\n=== 共识合约验证 ===")
    
    # 基础验证
    print(f"合约代码长度: {len(w3.eth.get_code(consensus_contract.address))} bytes")
    print(f"关联激励合约地址: {consensus_contract.functions.incentiveContract().call()} (正确性: {consensus_contract.functions.incentiveContract().call() == incentive_address})")
    
    # 参数验证
    print(f"最小参与者数: {consensus_contract.functions.minParticipants().call()}")
    print(f"模型超时时间: {consensus_contract.functions.modelUpdateTimeout().call()}秒")
    
    # 初始状态验证
    assert consensus_contract.functions.flRound().call() == 0, "初始轮次应为0"
    assert not consensus_contract.functions.isFLActive().call(), "初始应未激活"

# 执行验证
validate_consensus_deployment(consensus, incentive.address)

In [ ]:
results = []
def register_test_participant():
    user = w3.eth.accounts[random.randint(0, 9)]  # 使用随机账户
    local_data_cat = random.choice([0, 1, 2])  # 0=低, 1=中, 2=高
    local_resource_cat = random.choice([0, 1, 2, 3])  # 0=低, 1=中, 2=高, 3=超高

    # 选择最接近的合约项
    data_cat = min(local_data_cat, 2)  # 确保不超过最高类别
    resource_cat = min(local_resource_cat, 2)  # 确保不超过最高类别

    stake = incentive.functions.stakeMatrix(data_cat, resource_cat).call()

    try:
        # 用户加入合约
        tx_hash = incentive.functions.join(data_cat, resource_cat).transact({
            'from': user,
            'value': stake
        })
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

        # 记录结果
        results.append({
            'user': user,
            'local_data_cat': local_data_cat,
            'local_resource_cat': local_resource_cat,
            'chosen_data_cat': data_cat,
            'chosen_resource_cat': resource_cat,
            'stake': Web3.from_wei(stake, 'ether'),
            'status': 'success',
            'tx_hash': tx_hash.hex()
        })
    except Exception as e:
        results.append({
            'user': user,
            'local_data_cat': local_data_cat,
            'local_resource_cat': local_resource_cat,
            'chosen_data_cat': data_cat,
            'chosen_resource_cat': resource_cat,
            'stake': Web3.from_wei(stake, 'ether'),
            'status': 'failed',
            'error': str(e)
        })

In [ ]:

def validate_state(expected_phase, expected_participants, auto_closed=False):
    current_phase = incentive.functions.currentPhase().call()
    actual_participants = incentive.functions.getParticipantCount().call()
    
    print(f"当前阶段: {['ACTIVE','GRACE','CLOSED'][current_phase]}")
    print(f"参与者数量: {actual_participants}/{incentive.functions.minParticipants().call()}")
    
    assert current_phase == ["ACTIVE","GRACE","CLOSED"].index(expected_phase)
    assert actual_participants == expected_participants
    if auto_closed:
        assert incentive.functions.autoClosed().call()

#### 部署consensus合约并配置参数

### 模拟实验开始了

#### 模拟用户参与

所有的参与者

In [ ]:
# 模拟用户参与
num_users = 10
results = []

for i in range(num_users):
    user = w3.eth.accounts[i + 10]  # 使用不同账户
    
    # 随机生成用户的本地数据类别和资源类别
    local_data_cat = random.choice([0, 1, 2])  # 0=低, 1=中, 2=高
    local_resource_cat = random.choice([0, 1, 2,3])  # 0=低, 1=中, 2=高
    
    # 选择最接近的合约项
    data_cat = min(local_data_cat, 2)  # 确保不超过最高类别
    resource_cat = min(local_resource_cat, 2)  # 确保不超过最高类别
    
    stake = incentive.functions.stakeMatrix(data_cat, resource_cat).call()
    
    try:
        # 用户加入合约
        tx_hash = incentive.functions.join(data_cat, resource_cat).transact({
            'from': user,
            'value': stake
        })
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        
        # 记录结果
        results.append({
            'user': user,
            'local_data_cat': local_data_cat,
            'local_resource_cat': local_resource_cat,
            'chosen_data_cat': data_cat,
            'chosen_resource_cat': resource_cat,
            'stake': Web3.from_wei(stake, 'ether'),
            'status': 'success',
            'tx_hash': tx_hash.hex()
        })
    except Exception as e:
        results.append({
            'user': user,
            'local_data_cat': local_data_cat,
            'local_resource_cat': local_resource_cat,
            'chosen_data_cat': data_cat,
            'chosen_resource_cat': resource_cat,
            'stake': Web3.from_wei(stake, 'ether'),
            'status': 'failed',
            'error': str(e)
        })

# 显示参与结果
df = pd.DataFrame(results)
display(Markdown("### 用户参与结果"))
display(df)

#### 参与成功的用户进入FL consensus

In [ ]:
# 导入必要库
import pandas as pd
from IPython.display import display, Markdown
from web3 import Web3
import random

# 筛选成功参与者
success_df = df[df['status'] == 'success']
valid_participants = success_df['user'].tolist()

display(Markdown("### 有效参与者列表"))
display(pd.DataFrame({
    '地址': valid_participants,
    '选择的数据类别': success_df['chosen_data_cat'],
    '选择的资源类别': success_df['chosen_resource_cat']
}))

# 验证参与者资格
def validate_participant(addr):
    """验证参与者资格"""
    participant_info = incentive.functions.participants(addr).call()
    return {
        '地址': addr,
        '已加入': participant_info[5],
        '已奖励': participant_info[4],
        '质押金额': Web3.from_wei(participant_info[2], 'ether'),
        '加入时间': participant_info[3]
    }

validation_results = [validate_participant(addr) for addr in valid_participants]
display(Markdown("### 链上资格验证"))
display(pd.DataFrame(validation_results))



In [ ]:
def validate_qualified_participants():
    """多轮计算后验证合格参与者"""
    print("\n=== 多轮资格验证 ===")
    
    # 阶段验证
    current_phase = incentive.functions.currentPhase().call()
    assert current_phase == 2, f"合约必须处于CLOSED阶段，当前阶段：{['ACTIVE','GRACE','CLOSED'][current_phase]}"
    
    # 获取所有参与者
    all_participants = incentive.functions.participantList().call()
    validation_results = []
    
    # 详细验证每个参与者
    for addr in all_participants:
        p = incentive.functions.participants(addr).call()
        multiplier = incentive.functions.multiplierMatrix(p[0], p[1]).call()
        pending = incentive.functions.pendingRewards(addr).call()
        
        # 验证标准
        is_qualified = p[4]  # isRewarded
        validation_results.append({
            '地址': addr[-6:],
            '数据类别': p[0],
            '资源类别': p[1],
            '乘数': multiplier,
            '质押金额': Web3.from_wei(p[2], 'ether'),
            '加入时间': p[3],
            '是否逾期': p[3] > incentive.functions.timeConfig().call()[0] - 3600,
            '待领取奖励': Web3.from_wei(pending, 'ether'),
            '合格状态': '是' if is_qualified else '否',
            '失败原因': _get_failure_reason(p, multiplier) if not is_qualified else ''
        })
    
    # 生成报告
    report_df = pd.DataFrame(validation_results)
    display(report_df)
    
    # 汇总统计
    qualified_count = report_df[report_df['合格状态'] == '是'].shape[0]
    total_rewards = sum(report_df['待领取奖励'])
    print(f"\n合格参与者总数: {qualified_count}")
    print(f"总待发放奖励: {total_rewards} ETH")
    print(f"合约当前余额: {Web3.from_wei(incentive.functions.currentPoolBalance().call(), 'ether')} ETH")

def _get_failure_reason(p, multiplier):
    """生成资格失败原因"""
    reasons = []
    
    # 基础验证
    if not p[5]:  # joined字段
        return "未成功注册"
    if p[3] > incentive.functions.timeConfig().call()[0]:  # 超过截止时间
        reasons.append("逾期注册")
    if multiplier == 0:
        reasons.append("零乘数组合")
    
    # 惩罚验证
    if p[3] > incentive.functions.timeConfig().call()[0] - 3600:
        reasons.append("最后1小时加入")
    
    return ",".join(reasons) if reasons else "未知原因"

# 执行验证
validate_qualified_participants()

In [ ]:
def validate_incentive_state():
    phase = incentive.functions.currentPhase().call()
    deadline = incentive.functions.timeConfig().call()[0]
    current_time = w3.eth.get_block('latest').timestamp
    
    print("=== 激励合约状态验证 ===")
    print(f"当前阶段: {['ACTIVE', 'GRACE', 'CLOSED'][phase]}")
    print(f"当前时间: {current_time}")
    print(f"注册截止时间: {deadline} ({'已过期' if current_time > deadline else '有效'})")
    
    # 获取所有已加入的参与者
    all_participants = incentive.functions.participantList().call()
    print(f"总参与者数: {len(all_participants)}")
    
    # 验证有效参与者（joined=True）
    valid_count = 0
    for addr in all_participants:
        participant_info = incentive.functions.participants(addr).call()
        if participant_info[5]:  # joined字段
            valid_count += 1
            
    print(f"合格参与者数（已加入）: {valid_count} (应 ≥ 最小参与者数)")
    
    # 额外显示isRewarded状态
    rewarded_count = sum([incentive.functions.participants(addr).call()[4] for addr in all_participants])
    print(f"已奖励参与者数: {rewarded_count}")

validate_incentive_state()

In [ ]:
def deep_validate_incentive():
    # 获取完整时间配置
    deadline, grace_period, claim_window = incentive.functions.timeConfig().call()
    current_time = w3.eth.get_block('latest').timestamp
    
    print("=== 激励合约深度验证 ===")
    print(f"当前区块时间: {current_time}")
    print(f"注册截止时间: {deadline} (剩余时间: {deadline - current_time if current_time < deadline else '已过期'})")
    print(f"宽限期: {grace_period}秒 | 领取窗口: {claim_window}秒")
    
    # 验证finalize执行结果
    finalize_tx = incentive.functions.finalize().transact({'gas': 2_000_000})
    finalize_receipt = w3.eth.wait_for_transaction_receipt(finalize_tx)
    print(f"Finalize交易状态: {'成功' if finalize_receipt.status == 1 else '失败'}")

deep_validate_incentive()

In [ ]:
def validate_finalize():
    # 检查finalize是否已执行
    finalize_tx = incentive.functions.finalize().transact({'gas': 2_000_000})
    receipt = w3.eth.wait_for_transaction_receipt(finalize_tx)
    
    print(f"Finalize交易状态: {'成功' if receipt.status == 1 else '失败'}")
    print(f"Gas消耗: {receipt.gasUsed}")
    print(f"区块确认数: {receipt.blockNumber}")

validate_finalize()

In [ ]:
def safe_advance_to_closed():
    """安全推进到CLOSED阶段"""
    current_phase = incentive.functions.currentPhase().call()
    
    if current_phase == 0:
        # 确保时间推进足够
        deadline = incentive.functions.timeConfig().call()[0]
        current_time = w3.eth.get_block('latest').timestamp
        if current_time < deadline:
            time_diff = deadline - current_time + 1
            w3.provider.make_request('evm_increaseTime', [time_diff])
            w3.provider.make_request('evm_mine', [])
        
        # 执行finalize
        try:
            tx_hash = incentive.functions.finalize().transact({'gas': 1_000_000})
            receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            assert receipt.status == 1, "Finalize交易失败"
        except Exception as e:
            print(f"Finalize失败: {str(e)}")
            raise
    
    if incentive.functions.currentPhase().call() == 1:
        grace_period = incentive.functions.timeConfig().call()[1]
        w3.provider.make_request('evm_increaseTime', [grace_period + 1])
        w3.provider.make_request('evm_mine', [])

    print(f"最终阶段: {['ACTIVE', 'GRACE', 'CLOSED'][incentive.functions.currentPhase().call()]}")

safe_advance_to_closed()

In [ ]:
def debug_consensus():
    """调试共识合约关键函数"""
    print("\n=== 共识合约调试 ===")
    
    # 验证基础状态
    print(f"FL激活状态: {consensus.functions.isFLActive().call()}")
    print(f"当前轮次: {consensus.functions.flRound().call()}")
    
    # 分步调试getEligibleParticipants
    try:
        print("尝试获取合格参与者...")
        eligible = consensus.functions.getEligibleParticipants().call()
        print(f"成功获取 {len(eligible)} 个合格参与者")
    except Exception as e:
        print(f"调用失败: {str(e)}")
        print("可能原因: ")
        print("1. 激励合约未处于CLOSED阶段")
        print("2. 没有符合条件的参与者")
        print("3. 函数存在逻辑错误")

debug_consensus()